In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from mp_api.client import MPRester
from pymatgen.analysis.structure_matcher import StructureMatcher
from monty.serialization import loadfn, dumpfn

/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mpr = MPRester()

In [4]:
candidates = mpr.materials.summary.search(theoretical=False, elements=['O'], energy_above_hull=(0,0))

Retrieving SummaryDoc documents: 100%|█████████████████████████████████████████████| 7122/7122 [00:34<00:00, 207.58it/s]


In [5]:
from pymatgen.core import Structure

# Define f-block elements (lanthanides + actinides)
lanthanides = [
    "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu",
    "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"
]

actinides = [
    "Ac", "Th", "Pa", "U", "Np", "Pu", "Am",
    "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr"
]

non_TM = ["Zn", "Cd", "Hg"]

f_block = set(lanthanides + actinides + non_TM)

def filter_out_fblock(structures):
    """Remove any structures containing f-block elements."""
    filtered = []
    for s in structures:
        elems = {str(el) for el in s.composition.elements}
        if not elems.intersection(f_block):
            filtered.append(s)
    return filtered

In [6]:
filtered_candidates = filter_out_fblock(candidates)

In [7]:
from pymatgen.analysis.structure_matcher import ElementComparator
from pymatgen.core import Element, Composition
class TMComparator(ElementComparator):
    def are_equal(self, el1: Element, el2: Element):
        if el1 == el2:
            return True
        elif (
            (el1.contains_element_type('transition_metal') 
             and el2.contains_element_type('metal') 
             and not el2.contains_element_type('transition_metal'))
        ): # make sure 1 element is TM, other is metal that's not a TM
            for el in el2.elements:
                if 'd' not in el.electronic_structure: # make sure metal has no d-electrons
                    return True
        elif (el1.contains_element_type('metal') 
             and not el1.contains_element_type('transition_metal') 
             and el2.contains_element_type('transition_metal')
        ):  # make sure 1 element is TM, other is metal that's not a TM
            for el in el2.elements:
                if 'd' not in el.electronic_structure:  # make sure metal has no d-electrons
                    return True
        elif el1.contains_element_type('transition_metal') or el2.contains_element_type('transition_metal'):
            return False
        
        return False
    def get_hash(self, composition):
        """Get the fractional composition."""
        comphash = composition.fractional_composition.formula
        elements = composition.elements
        tm_count = 0
        metal_count = 0
        for el in elements:
            if el.is_transition_metal and tm_count == 0:
                comphash = comphash.replace(el.symbol, 'J')
                tm_count += 1 # make sure only 1 TM in whole material
            elif el.is_metal and metal_count == 0:
                if 'd' not in el.electronic_structure:
                    comphash = comphash.replace(el.symbol, 'J')
                    metal_count += 1
        return comphash

sm = StructureMatcher(comparator=TMComparator())

In [32]:
matching_pairs = {}

In [34]:
for candidate in filtered_candidates:
    for candidate2 in filtered_candidates:
        if candidate.composition.anonymized_formula == candidate2.composition.anonymized_formula:
            if candidate.composition.formula != candidate2.composition.formula:
                if sm.fit(candidate.structure, candidate2.structure):
                    elements = candidate.elements + candidate2.elements
                    for element in elements:
                        if element.is_transition_metal:
                            dopant = element
                    if candidate.composition.contains_element_type('transition_metal'):
                        for element in candidate2.elements:
                            if element not in candidate.elements:
                                metal = element.symbol
                        if candidate2.material_id in previous.keys():
                            if previous[candidate2.material_id]['dopant'] == dopant.symbol:
                                continue
                        key = candidate2.material_id + dopant.symbol
                        matching_pairs[key] = {
                            'matches': (candidate.material_id, candidate.formula_pretty),
                            'formula': candidate2.formula_pretty,
                            'structure': candidate2.structure,
                            'dopant' : dopant.symbol,
                            'metal': metal
                        }
                    elif candidate2.composition.contains_element_type('transition_metal'):
                        for element in candidate.elements:
                            if element not in candidate2.elements:
                                metal = element.symbol
                        if candidate.material_id in previous.keys():
                            if previous[candidate.material_id]['dopant'] == dopant.symbol:
                                continue
                        key = candidate.material_id + dopant.symbol
                        matching_pairs[key] = {
                            'matches': (candidate2.material_id, candidate2.formula_pretty),
                            'formula': candidate.formula_pretty,
                            'structure': candidate.structure,
                            'dopant' : dopant.symbol,
                            'metal':metal
                        }

In [36]:
dumpfn(matching_pairs, '../../data/matching_extended_screening.json')